In [1]:
__author__ = "Yicheng Li"
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
import random
from sklearn import preprocessing
import tensorflow as tf

dir_path = ''
df = pd.read_pickle(dir_path+'df_5min.pkl')
df = df.dropna()

df.head()

/home/shared/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


,USDT_BTC_close,USDT_BTC_high,USDT_BTC_low,USDT_BTC_open,USDT_BTC_quoteVolume,USDT_BTC_volume,USDT_BTC_weightedAverage,BTC_ETH_close,BTC_ETH_high,BTC_ETH_low,...,BTC_LTC_quoteVolume,BTC_LTC_volume,BTC_LTC_weightedAverage,BTC_XRP_close,BTC_XRP_high,BTC_XRP_low,BTC_XRP_open,BTC_XRP_quoteVolume,BTC_XRP_volume,BTC_XRP_weightedAverage
time,,,,,,,,,,,,,,,,,,,,,
2016-01-01 08:00:00,433.010137,433.010137,433.010137,433.010137,0.0,0.0,433.010137,0.002174,0.002177,0.002174,...,1.4,0.011309,0.008078,0.000014,0.000014,0.000014,0.000014,0.000000,0.000000,0.000014
2016-01-01 08:05:00,433.010137,433.010137,433.010137,433.010137,0.0,0.0,433.010137,0.002177,0.002177,0.002175,...,0.0,0.000000,0.008079,0.000014,0.000014,0.000014,0.000014,980.096150,0.013429,0.000014
2016-01-01 08:10:00,433.010137,433.010137,433.010137,433.010137,0.0,0.0,433.010137,0.002175,0.002175,0.002175,...,0.0,0.000000,0.008079,0.000014,0.000014,0.000014,0.000014,657.512017,0.009028,0.000014
2016-01-01 08:15:00,433.010137,433.010137,433.010137,433.010137,0.0,0.0,433.010137,0.002174,0.002175,0.002174,...,0.0,0.000000,0.008079,0.000014,0.000014,0.000014,0.000014,862.028200,0.011801,0.000014
2016-01-01 08:20:00,433.010137,433.010137,433.010137,433.010137,0.0,0.0,433.010137,0.002174,0.002174,0.002174,...,0.0,0.000000,0.008079,0.000014,0.000014,0.000014,0.000014,5331.418950,0.072934,0.000014


In [2]:
scaler_price = preprocessing.MinMaxScaler(feature_range=(0.1, 1))
scaler_features = preprocessing.MinMaxScaler(feature_range=(0.1, 1))

In [3]:
# function to create train, validation, test data given sequence length
def load_data(df, seq_len):
    labels = df['USDT_BTC_weightedAverage'].as_matrix()
    feature_set = [6]
    
    data_raw = df.as_matrix() # convert to numpy array
#     data_raw = preprocessing.minmax_scale(data_raw[:, feature_set]) # normalizing features
    scaler_price.fit(data_raw[:, [6]])
    data_raw = scaler_features.fit_transform(data_raw[:, feature_set])
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - seq_len + 1): 
        data.append(data_raw[index: index + seq_len, :])
    
    data = np.array(data)
    
    n_train_valid_pairs = 3
    each_train_set_size_pct = 25
    each_valid_set_size_pct = 5
    
    each_train_set_size = round(each_train_set_size_pct/100*data.shape[0])
    each_valid_set_size = round(each_valid_set_size_pct/100*data.shape[0])
    
    x_train_sets = []
    y_train_sets = []
    x_valid_sets = []
    y_valid_sets = []
    used = 0
    
    for i in range(n_train_valid_pairs):
        x_train_sets.append(data[used : used + each_train_set_size,:-1,:]) # cannot see last day, which we aim to predict
        y_train_sets.append(labels[used + seq_len-1 : used + each_train_set_size + seq_len-1])
        used += each_train_set_size
    
        x_valid_sets.append(data[used : used + each_valid_set_size,:-1,:])
        y_valid_sets.append(labels[used + seq_len-1 : used + each_valid_set_size + seq_len-1])
        used += each_valid_set_size
    
    x_test = data[used : , :-1, :]
    y_test = labels[seq_len-1 + used : ]
    
    x_train = np.concatenate(x_train_sets, axis=0)
    y_train = np.concatenate(y_train_sets, axis=0)
    x_valid = np.concatenate(x_valid_sets, axis=0)
    y_valid = np.concatenate(y_valid_sets, axis=0)
    
    return [x_train, y_train, x_valid, y_valid, x_test, y_test]

In [4]:
# create train, test data
seq_len = 5 # choose sequence length
x_train, y_train, x_valid, y_valid, x_test, y_test = load_data(df, seq_len)
y_train = y_train.reshape([-1,1])
y_valid = y_valid.reshape([-1,1])
y_test = y_test.reshape([-1,1])
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ', y_train.shape)
print('x_valid.shape = ',x_valid.shape)
print('y_valid.shape = ', y_valid.shape)
print('x_test.shape = ', x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (180096, 4, 1)
y_train.shape =  (180096, 1)
x_valid.shape =  (36018, 4, 1)
y_valid.shape =  (36018, 1)
x_test.shape =  (24016, 4, 1)
y_test.shape =  (24016, 1)


In [7]:
# shuffle training data
s = np.arange(x_train.shape[0])
np.random.shuffle(s)
x_train = x_train[s]
y_train = y_train[s]

In [5]:
def SMAPE(y, y_pred):
    if len(y) != len(y_pred):
        raise ValueError('Length of prediction array is not equal to length of y array.')
    return np.mean(np.abs(y-y_pred)*2/(np.abs(y)+np.abs(y_pred)))

## baseline performance

In [9]:
y_pred = np.roll(y_valid,1, axis=0)
print('baseline dev_SMAPE=',SMAPE(y_valid[1:], y_pred[1:]))
y_pred = np.roll(y_test,1, axis=0)
y_pred[0] = y_valid[-1]
print('baseline test_SMAPE=',SMAPE(y_test, y_pred))

baseline dev_SMAPE= 0.0023080371393128284
baseline test_SMAPE= 0.002474303030669981


## LSTM model graph

In [6]:
index_in_epoch = 0;
perm_array  = np.arange(x_train.shape[0])
np.random.shuffle(perm_array)

# function to get the next batch
def get_next_batch(batch_size):
    global index_in_epoch, x_train, perm_array   
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > x_train.shape[0]:
        np.random.shuffle(perm_array) # shuffle permutation array
        start = 0 # start next epoch
        index_in_epoch = batch_size
        
    end = index_in_epoch
    return x_train[perm_array[start:end]], scaler_price.transform(y_train[perm_array[start:end]])

x_1000_train, y_1000_train = get_next_batch(1000) # special batch of 1000 records in training set

In [35]:
# parameters
n_steps = seq_len-1 
n_inputs = x_train.shape[-1]
n_neurons = 100  # cell.state_size
n_outputs = 1
n_layers = 2
# learning_rate = 5e-5
batch_size = 100
n_epochs = 200 
train_set_size = x_train.shape[0]
test_set_size = x_test.shape[0]
keep_prob = 1
max_gradient_norm = 5

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_outputs])

# use LSTM Cell
layers = [tf.contrib.rnn.LSTMCell(num_units=n_neurons, \
                                  initializer=tf.contrib.layers.xavier_initializer(), \
                                  activation=tf.nn.elu)
         for layer in range(n_layers)]

# use Basic LSTM Cell 
# layers = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, activation=tf.nn.elu)
#           for layer in range(n_layers)]

# use LSTM Cell with peephole connections
#layers = [tf.contrib.rnn.LSTMCell(num_units=n_neurons, 
#                                  activation=tf.nn.leaky_relu, use_peepholes = True)
#          for layer in range(n_layers)]

# use GRU cell
#layers = [tf.contrib.rnn.GRUCell(num_units=n_neurons, activation=tf.nn.leaky_relu)
#          for layer in range(n_layers)]

multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)

outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)
outputs = tf.nn.dropout(outputs, keep_prob)

# 'outputs' is a tensor of shape [batch_size, n_steps, n_neurons(cell.state_size)]
stacked_outputs = tf.reshape(outputs, [-1, n_neurons]) 
stacked_outputs = tf.layers.dense(stacked_outputs, n_outputs)
outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])
outputs = outputs[:,n_steps-1,:] # keep only last output of sequence

loss = tf.reduce_mean(tf.square(tf.div(outputs,y)-1)) # loss function = mean squared error 
# loss = tf.reduce_mean(tf.square(outputs - y)) # loss function = mean squared error 
#loss = tf.reduce_mean(tf.abs(y-outputs)*2/(tf.abs(y)+tf.abs(outputs))) # SMAPE
params = tf.trainable_variables()
gradients = tf.gradients(loss, params)
gradient_norm = tf.global_norm(gradients)
clipped_gradients, _ = tf.clip_by_global_norm(gradients, max_gradient_norm)
clipped_norm = tf.global_norm(clipped_gradients)
param_norm = tf.global_norm(params)
learning_rate_placeholder = tf.placeholder(tf.float32, [], name='learning_rate')
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate_placeholder) 
# training_op = optimizer.minimize(loss)
training_op = optimizer.apply_gradients(zip(clipped_gradients, params))

# initialize parameters
sess = tf.Session()
global_step = tf.Variable(0, name='global_step', trainable=False)
saver = tf.train.Saver(max_to_keep=2)

In [30]:
experiment_name = 'overfit_5min_LSTM_1_features_5seq_100neuron_2layer' # type your experiment name here before running the code below

In [39]:
# run graph
ckpt = tf.train.get_checkpoint_state(experiment_name)
v2_path = ckpt.model_checkpoint_path + ".index" if ckpt else ""
if ckpt and (tf.gfile.Exists(ckpt.model_checkpoint_path) or tf.gfile.Exists(v2_path)):
#     iteration = tf.get_variable('iteration',[1])
    saver.restore(sess, ckpt.model_checkpoint_path)
    iteration = global_step.eval(session=sess) # get last global_step
    print("Start from iteration:", iteration)
else:
    print('There is not saved parameters. Creating model with fresh parameters.')
#     iteration = tf.get_variable('iteration',[1], initializer = tf.zeros_initializer)
    sess.run(tf.global_variables_initializer())
    iteration = 0
#     sess.run(iteration)
    
old_loss = 1000
lr = 1e-4
while True:
    iteration = iteration + 1
    x_batch, y_batch = get_next_batch(batch_size) # fetch the next training batch 
     
    # train on this batch
    sess.run(training_op, feed_dict={X: x_batch, y: y_batch, learning_rate_placeholder:lr})
    
    if iteration % 500 == 0:
#             loss_train = loss.eval(feed_dict={X: x_train, y: y_train}) 
#             loss_valid = loss.eval(feed_dict={X: x_valid, y: y_valid}) 
        y_1000_train_pred, loss_val, param_norm_val = sess.run([outputs, loss, param_norm],\
                                                                feed_dict={X: x_1000_train, y:y_1000_train, \
                                                                          learning_rate_placeholder:lr})
        if loss_val > old_loss * 1.2:
            lr /= 2
        old_loss = loss_val
        
        y_valid_pred = scaler_price.inverse_transform(sess.run(outputs, feed_dict={X: x_valid}))
        print('%.2f epochs, iter %d: train_loss = %.9f, param_norm = %.3f, train_SMAPE/valid_SMAPE = %.6f/%.6f' \
              %(iteration*batch_size/train_set_size, iteration, loss_val, param_norm_val, \
                SMAPE(scaler_price.inverse_transform(y_1000_train), scaler_price.inverse_transform(y_1000_train_pred)), \
                SMAPE(y_valid, y_valid_pred)))
#             print('%.2f epochs: loss train/valid = %.6f/%.6f'%(
#                 iteration*batch_size/train_set_size, loss_train, loss_valid))
    if iteration % 1000 == 0:
        global_step.assign(iteration).eval(session=sess) # set and update(eval) global_step with index, i
        save_path = saver.save(sess, "./"+experiment_name+"/model.ckpt", global_step=global_step)
        print('Saved parameters to %s' % save_path)

y_train_pred = sess.run(outputs, feed_dict={X: x_train})
y_valid_pred = sess.run(outputs, feed_dict={X: x_valid})
y_test_pred = sess.run(outputs, feed_dict={X: x_test})

There is not saved parameters. Creating model with fresh parameters.
0.28 epochs, iter 500: train_loss = 0.000220844, param_norm = 20.906, train_SMAPE/valid_SMAPE = 0.048714/0.038795
0.56 epochs, iter 1000: train_loss = 0.000150334, param_norm = 20.910, train_SMAPE/valid_SMAPE = 0.040159/0.031119
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-1000
0.83 epochs, iter 1500: train_loss = 0.000078298, param_norm = 20.913, train_SMAPE/valid_SMAPE = 0.029041/0.022767
1.11 epochs, iter 2000: train_loss = 0.000131965, param_norm = 20.916, train_SMAPE/valid_SMAPE = 0.038045/0.028105
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-2000
1.39 epochs, iter 2500: train_loss = 0.000029608, param_norm = 20.916, train_SMAPE/valid_SMAPE = 0.017791/0.014474
1.67 epochs, iter 3000: train_loss = 0.000057133, param_norm = 20.917, train_SMAPE/valid_SMAPE = 0.025876/0.019881
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuro

14.44 epochs, iter 26000: train_loss = 0.000004854, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.003279/0.003749
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-26000
14.71 epochs, iter 26500: train_loss = 0.000004966, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.003397/0.003834
14.99 epochs, iter 27000: train_loss = 0.000004788, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.002950/0.003603
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-27000
15.27 epochs, iter 27500: train_loss = 0.000004829, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.003216/0.003715
15.55 epochs, iter 28000: train_loss = 0.000004798, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.003040/0.003636
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-28000
15.82 epochs, iter 28500: train_loss = 0.000005097, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.003773/0.004103
16.10 epochs, iter 29000: tr

28.60 epochs, iter 51500: train_loss = 0.000004831, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.003110/0.003666
28.87 epochs, iter 52000: train_loss = 0.000004801, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.003058/0.003675
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-52000
29.15 epochs, iter 52500: train_loss = 0.000004793, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.002932/0.003594
29.43 epochs, iter 53000: train_loss = 0.000004841, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.003215/0.003733
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-53000
29.71 epochs, iter 53500: train_loss = 0.000004868, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.003248/0.003751
29.98 epochs, iter 54000: train_loss = 0.000004945, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.003430/0.003861
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-54000
30.26 epochs, iter 54500: tr

42.75 epochs, iter 77000: train_loss = 0.000004776, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.002904/0.003598
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-77000
43.03 epochs, iter 77500: train_loss = 0.000004873, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.003090/0.003677
43.31 epochs, iter 78000: train_loss = 0.000004782, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.002962/0.003618
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-78000
43.59 epochs, iter 78500: train_loss = 0.000004829, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.003145/0.003717
43.87 epochs, iter 79000: train_loss = 0.000004790, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.002946/0.003612
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-79000
44.14 epochs, iter 79500: train_loss = 0.000004778, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.002969/0.003618
44.42 epochs, iter 80000: tr

56.91 epochs, iter 102500: train_loss = 0.000004806, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.003060/0.003649
57.19 epochs, iter 103000: train_loss = 0.000004809, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.003085/0.003670
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-103000
57.47 epochs, iter 103500: train_loss = 0.000004798, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.002997/0.003633
57.75 epochs, iter 104000: train_loss = 0.000004768, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.002912/0.003598
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-104000
58.02 epochs, iter 104500: train_loss = 0.000004773, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.002937/0.003604
58.30 epochs, iter 105000: train_loss = 0.000004780, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.002975/0.003624
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-105000
58.58 epochs, iter 

71.07 epochs, iter 128000: train_loss = 0.000004763, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.002949/0.003616
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-128000
71.35 epochs, iter 128500: train_loss = 0.000004795, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.002989/0.003613
71.63 epochs, iter 129000: train_loss = 0.000004764, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.002897/0.003597
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-129000
71.91 epochs, iter 129500: train_loss = 0.000004766, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.002899/0.003599
72.18 epochs, iter 130000: train_loss = 0.000004776, param_norm = 20.919, train_SMAPE/valid_SMAPE = 0.002920/0.003596
Saved parameters to ./overfit_5min_LSTM_1_features_5seq_100neuron_2layer/model.ckpt-130000
72.46 epochs, iter 130500: train_loss = 0.000004808, param_norm = 20.918, train_SMAPE/valid_SMAPE = 0.002919/0.003601
72.74 epochs, iter 

KeyboardInterrupt: 

In [40]:
# y_train_pred = scaler_price.inverse_transform(sess.run(outputs, feed_dict={X: x_train}))
# print(SMAPE(y_train, y_train_pred))

y_valid_pred = scaler_price.inverse_transform(sess.run(outputs, feed_dict={X: x_valid}))
print(SMAPE(y_valid, y_valid_pred))

y_test_pred = scaler_price.inverse_transform(sess.run(outputs, feed_dict={X: x_test}))
print(SMAPE(y_test, y_test_pred))

0.003770190121577454
0.004232223008743406
